# Colorspace - Patterns - Pixel Screens

Create png files based on the screen dump from the atari800 emulation of colourspace.

Read in the ram history from our dump file. Use the same file as the volumetric graph notebook.

In [1]:
log_file = "colorspace_patterns_volumetric.txt"
input_file = open(log_file,'r')

flatten = lambda l: [e for sublist in l for e in sublist]

ram_history = []
lines = input_file.readlines()
for i in range(1, len(lines), 131):
    # Screen RAM (7000-8000)
    raw_ram = [l[6:126].split() for l in lines[i:i+104]]
    pixel_ram = [int(v,16) for l in raw_ram for v in l][:0x1000]
    # Display list (8000-9000)
    raw_ram = [l[6:126].split() for l in lines[i+103:i+127]]
    display_list = [int(v,16) for l in raw_ram for v in l]
    # Color Registers
    color_registers = [int(i,16) for i in lines[i+129].split()[1:10]]
    ram_history += [(pixel_ram, color_registers, display_list)]

Helper functions for implementing the display list entries

In [2]:
def getByteForAddress(address,scr):
    if 0x7000 <= address < 0x8000:
        return scr[address - 0x7000]
    return 0

def drawBlankLines(pixels, byte, y, num, scr):
    #pixels[x, y] = ImageColor.getrgb('#000000')
    return num

def drawScanLine(pixels, byte, y, num, scr):
    for x in range(0,40):
        byte = getByteForAddress(address+x, scr)
        left_nibble = byte >> 4 & 0x0F 
        right_nibble = byte & 0x0F
        for n in range(0,4):
            pixels[y][(x*8)+n] = col_register[left_nibble]
        for n in range(0,4):
            pixels[y][(x*8)+n+4] = col_register[right_nibble]
    return 1

dlist_dict = {
    0x70: (drawBlankLines,1,1),
    0x90: (drawBlankLines,1,1),
    0x46: (drawBlankLines,3,1),
    0x4F: (drawScanLine,  3,1),
}
def display_list_entry(dl):
    # Consume entries in the display list one by one.
    i = 0
    while i < len(dl):
        b = dl[i]
        if b not in dlist_dict:
            yield (None,None,None,None,None)
        draw, advance, lines = dlist_dict[b]
        address = (dl[i+2]<<8 | dl[i+1]) if advance > 1 else 0
        i += advance
        yield (b, draw, address, lines,i)


Read in the ram  history and paint a png file using the display lists and the pixel data.

In [3]:
ROWS = 192
COLS = 320

pixel_history = []
#selected = [ram_history[x] for x in [188,236]]
for j, (scr, col_register, display_list) in enumerate(ram_history):
    pixels = [[0 for i in range(0,COLS)] for i in range(0,ROWS)]
    dlist = display_list_entry(display_list)
    x,y = 0,0
    while True:
        token, draw, address, num, i = next(dlist)
        if draw == None:
            break
        y += draw(pixels, address, y, num, scr)
    pixel_history+= [pixels]

In [23]:
flatten = lambda l: [e for sublist in l for e in sublist]

pattern_breaks = []
for i,h in enumerate(pixel_history):
    num_pixels = sum(flatten(h))
    if num_pixels > 1:
        continue
    pattern_breaks += [i]

patterns = []
pp = 0
for p in pattern_breaks:
    patterns += [pixel_history[pp:p]]
    pp = p
len(patterns)

34

In [21]:
# Fix up the patterns
del patterns[2]
del patterns[3]
del patterns[4]
del patterns[5]
del patterns[7]
del patterns[9]
del patterns[11]
del patterns[13]


In [16]:
def firstNonZero(l):
    for i,e in enumerate(l):
        if e > 0:
            return i
    return i

def getLeftMostPixelForPattern(pattern):
    return min([firstNonZero(r) for scr in pattern for r in scr])

def getRightMostPixelForPattern(pattern):
    return len(pattern[0][0]) - min([firstNonZero(list(reversed(r)))
                     for scr in pattern for r in scr])

def rotatePatternToCols(pattern):
    pattern_cols = []
    for scr in pattern:
        col_arr = [[0 for i in range(len(scr))] for j in range(len(scr[0]))]
        for i,r in enumerate(scr):
            for j,v in enumerate(r):
                col_arr[j][i] = v
        pattern_cols += [col_arr]
    return pattern_cols

def cullPattern(pattern):
    first_left = getLeftMostPixelForPattern(pattern)
    last_right = getRightMostPixelForPattern(pattern)
    width = last_right - first_left

    pattern_as_cols = rotatePatternToCols(pattern)
    first_top = getLeftMostPixelForPattern(pattern_as_cols)
    last_bottom = getRightMostPixelForPattern(pattern_as_cols)
    height = last_bottom - first_top
    
    culled = [[r[first_left:last_right] 
              for r in scr[first_top:last_bottom]]
              for scr in pattern]
    return culled



In [17]:
from PIL import Image, ImageColor
import colorspace_colors as cc


In [22]:
# Select our pattern
import os

for i, pattern in enumerate(patterns):
    cull_pattern = cullPattern(pattern)
    for j, screen in enumerate(cull_pattern):
        cols=len(screen[0])
        rows=len(screen)
        img = Image.new( 'RGB', (cols,rows), "black")
        pixels = img.load()
        for y, row in enumerate(screen):
            for x, col in enumerate(row):
                # Skip black, white and gray cells
                if col in [0,1,0x0c]:
                    continue
                color =  cc.color_value_to_html[col]
                pixels[x, y] = ImageColor.getrgb(color)
        img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
        img.save(f"colorspace_patterns/pixels/pixel_pattern{i}_{j}.png")
